In [1]:
import pandas as pd
import numpy as np
import os
import math

import matplotlib.pyplot as plt
import seaborn as sns
import env
from sklearn.impute import SimpleImputer


import wrangle_zillow

In [2]:
df = wrangle_zillow.zillow_data()


In [6]:
df.head()


,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,None,None,Central,Condominium,None,None


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77575 entries, 0 to 77574
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77575 non-null  int64  
 1   parcelid                      77575 non-null  int64  
 2   airconditioningtypeid         25006 non-null  float64
 3   architecturalstyletypeid      206 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   77575 non-null  float64
 6   bedroomcnt                    77575 non-null  float64
 7   buildingclasstypeid           15 non-null     float64
 8   buildingqualitytypeid         49809 non-null  float64
 9   calculatedbathnbr             76960 non-null  float64
 10  decktypeid                    614 non-null    float64
 11  finishedfloor1squarefeet      6035 non-null   float64
 12  calculatedfinishedsquarefeet  77375 non-null  float64
 13  f

In [8]:
for col in df.columns:
    if col != 'id':
        print(col)
        print(df[col].value_counts())


parcelid
12612211    3
11991059    3
14254548    2
12099888    2
11921077    2
           ..
11994786    1
14187514    1
11446758    1
14013599    1
12826780    1
Name: parcelid, Length: 77381, dtype: int64
airconditioningtypeid
1.0     23185
13.0     1575
5.0       167
11.0       53
9.0        26
Name: airconditioningtypeid, dtype: int64
architecturalstyletypeid
7.0     172
8.0      19
21.0      7
2.0       5
3.0       3
Name: architecturalstyletypeid, dtype: int64
basementsqft
900.0     2
700.0     2
100.0     2
640.0     2
515.0     2
273.0     2
912.0     2
314.0     1
819.0     1
1809.0    1
604.0     1
126.0     1
588.0     1
786.0     1
669.0     1
800.0     1
396.0     1
112.0     1
645.0     1
384.0     1
1969.0    1
252.0     1
600.0     1
1218.0    1
280.0     1
405.0     1
224.0     1
512.0     1
380.0     1
204.0     1
200.0     1
90.0      1
3112.0    1
674.0     1
300.0     1
1252.0    1
254.0     1
3560.0    1
352.0     1
168.0     1
394.0     1
1416.0    1
38.0      1


In [9]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing.sort_values(by='num_rows_missing', ascending=False)


In [10]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 70

In [11]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
buildingclassdesc,77560,99.980664
buildingclasstypeid,77560,99.980664
finishedsquarefeet13,77533,99.945859
storytypeid,77525,99.935546
basementsqft,77525,99.935546
storydesc,77525,99.935546
yardbuildingsqft26,77505,99.909765
fireplaceflag,77403,99.778279
architecturalstyletypeid,77369,99.734451
architecturalstyledesc,77369,99.734451


In [12]:
df.taxdelinquencyflag.value_counts()


Y    2900
Name: taxdelinquencyflag, dtype: int64

In [13]:
def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prcnt_miss})
    rows_missing = df.merge(rows_missing,
                        left_index=True,
                        right_index=True)[['parcelid', 'num_cols_missing', 'percent_cols_missing']]
    return rows_missing.sort_values(by='num_cols_missing', ascending=False)

In [14]:
nulls_by_row(df)


,parcelid,num_cols_missing,percent_cols_missing
55648,14341728,48,70.588235
13457,167686999,48,70.588235
41667,167687839,48,70.588235
61283,17188862,47,69.117647
14547,167687739,47,69.117647
...,...,...,...
15560,17097816,24,35.294118
7294,14140297,24,35.294118
39386,14371475,24,35.294118
32896,14465914,23,33.823529


In [15]:
def summarize(df):
    '''
    This function will take in a single argument (a pandas dataframe) and 
    output to console various statistices on said dataframe, including:
    # .head()
    # .info()
    # .describe()
    # value_counts()
    # observation of nulls in the dataframe
    '''
    print('----------------------')
    print('Dataframe head')
    print(df.head(3))
    print('----------------------')
    print('Dataframe Info ')
    print(df.info())
    print('----------------------')
    print('Dataframe Description')
    print(df.describe())
    print('----------------------')
    num_cols = [col for col in df.columns if df[col].dtype != 'object']
    cat_cols = [col for col in df.columns if col not in num_cols]
    print('----------------------')
    print('Dataframe value counts ')
    for col in df.columns:
        if col in cat_cols:
            print(df[col].value_counts())
        else:
            # define bins for continuous columns and don't sort them
            print(df[col].value_counts(bins=10, sort=False))
    print('----------------------')
    print('nulls in df by column')
    print(nulls_by_col(df))
    print('----------------------')
    print('null in df by row')
    print(nulls_by_row(df))
    print('----------------------')

In [16]:
summarize(df)


----------------------
Dataframe head
        id  parcelid  airconditioningtypeid  architecturalstyletypeid  \
0  1727539  14297519                    NaN                       NaN   
1  1387261  17052889                    NaN                       NaN   
2    11677  14186244                    NaN                       NaN   

   basementsqft  bathroomcnt  bedroomcnt  buildingclasstypeid  \
0           NaN          3.5         4.0                  NaN   
1           NaN          1.0         2.0                  NaN   
2           NaN          2.0         3.0                  NaN   

   buildingqualitytypeid  calculatedbathnbr  decktypeid  \
0                    NaN                3.5         NaN   
1                    NaN                1.0         NaN   
2                    NaN                2.0         NaN   

   finishedfloor1squarefeet  calculatedfinishedsquarefeet  \
0                       NaN                        3100.0   
1                    1465.0                      

In [17]:
def remove_columns(df, cols_to_remove):
    df = df.drop(columns=cols_to_remove)
    return df

In [18]:
df.parcelid.value_counts()


12612211    3
11991059    3
14254548    2
12099888    2
11921077    2
           ..
11994786    1
14187514    1
11446758    1
14013599    1
12826780    1
Name: parcelid, Length: 77381, dtype: int64

In [19]:
df.unitcnt.isnull().sum()


26872

In [20]:
df.bedroomcnt.value_counts()


3.0     30436
2.0     19222
4.0     17549
5.0      4550
1.0      3391
6.0      1000
0.0       836
8.0       253
7.0       208
9.0        70
10.0       31
12.0       16
11.0        9
13.0        2
14.0        1
16.0        1
Name: bedroomcnt, dtype: int64

In [21]:
def get_upper_outliers(s, k=1.5):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, 0.75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

In [22]:
def add_upper_outlier_columns(df, k=1.5):
    '''
    Add a column with the suffix _outliers for all the numeric columns
    in the given dataframe.
    '''
    for col in df.select_dtypes('float64'):
        df[col + '_outliers_upper'] = get_upper_outliers(df[col], k)
    return df

In [23]:
df = add_upper_outlier_columns(df)
